In [ ]:
import optuna
from optuna import Trial
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
from ctgan.synthesizers.ctgan import CTGAN,EnhancedCTGAN
from scipy.stats import wasserstein_distance, entropy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def objective(trial: Trial, real_data, discrete_columns):

    params = {
        "dropout": trial.suggest_float("dropout", 0.0, 0.3),   
        "num_first": trial.suggest_categorical("num_first", [True, False]),
        "batch_first": trial.suggest_categorical("batch_first", [True, False]), 
        "num_layers": trial.suggest_int("num_layers", 2, 8),    
        "num_heads": trial.suggest_categorical("num_heads", [4, 8]),
    }
    metrics = {}
    try:
        ctgan = EnhancedCTGAN(
            embedding_dim=128,
            generator_dim=(256, 256),
            discriminator_dim=(256, 256),
            dropout=params["dropout"],
            num_first=params["num_first"],
            batch_first=params["batch_first"],
            num_layers=params["num_layers"],
            num_heads=params["num_heads"],
            epochs=20,
            generator_lr=2e-4,
            discriminator_lr=2e-4,
            discriminator_steps=1,  
            verbose=False,  
            cuda=True 
        )

        ctgan.fit(real_data, discrete_columns=discrete_columns)

        synthetic_data = ctgan.sample(len(real_data))

        metrics = evaluate_data_quality(real_data, synthetic_data, discrete_columns)
   
        trial.set_user_attr("metrics", metrics)
    
        composite_score = (
            0.3 * (1 - metrics["wd"]) + 
            0.2 * (1 - metrics["jsd"]) +
            0.2 * (1 - metrics["corr_diff"]) +
            0.3 * (1 - abs(metrics["auc"] - 0.5))
        )
        return composite_score

    except Exception as e:
        print(f"Trial failed with parameters: {params}. Error: {str(e)}")
        return 0  

def evaluate_data_quality(real, synthetic, discrete_columns):
    metrics = {}
    
    cont_cols = [col for col in real.columns if col not in discrete_columns]
    wd_scores = [
        wasserstein_distance(real[col], synthetic[col]) 
        for col in cont_cols
    ]
    metrics["wd"] = np.mean(wd_scores)
    
    jsd_scores = []
    for col in discrete_columns:
        real_p = real[col].value_counts(normalize=True).sort_index()
        syn_p = synthetic[col].value_counts(normalize=True).sort_index()
        all_cats = list(set(real_p.index) | set(syn_p.index))
        

        real_p = real_p.reindex(all_cats, fill_value=0).values + 1e-10
        syn_p = syn_p.reindex(all_cats, fill_value=0).values + 1e-10
        
 
        real_p /= real_p.sum()
        syn_p /= syn_p.sum()
        
        m = 0.5 * (real_p + syn_p)
        jsd = 0.5 * (entropy(real_p, m) + entropy(syn_p, m))
        jsd_scores.append(jsd)
    metrics["jsd"] = np.mean(jsd_scores)
    

    if len(cont_cols) > 0:
        real_corr = real[cont_cols].corr().values
        syn_corr = synthetic[cont_cols].corr().values
        metrics["corr_diff"] = np.linalg.norm(real_corr - syn_corr, ord="fro")
    else:
        metrics["corr_diff"] = 0.0

    combined = pd.concat([real, synthetic])
    labels = np.concatenate([np.ones(len(real)), np.zeros(len(synthetic))])
    
    X_train, X_test, y_train, y_test = train_test_split(
        combined, labels, test_size=0.3, stratify=labels
    )
    
    clf = RandomForestClassifier(n_estimators=100, random_state=0)
    clf.fit(X_train, y_train)
    preds = clf.predict_proba(X_test)[:, 1]
    metrics["auc"] = roc_auc_score(y_test, preds)
    
    return metrics
def visualize_study(study):

    fig = optuna.visualization.plot_param_importances(study)
    fig.show()
    

    fig = optuna.visualization.plot_parallel_coordinate(study)
    fig.show()
    

    df = study.trials_dataframe()
    metrics = [trial.user_attrs["metrics"] for trial in study.trials]

    df["wd"] = [m["wd"] for m in metrics]
    df["jsd"] = [m["jsd"] for m in metrics]
    df["corr_diff"] = [m["corr_diff"] for m in metrics]
    df["auc"] = [m["auc"] for m in metrics]
    
    params_to_plot = ["dropout", "num_layers"]
    param_cols = ["params_" + p for p in params_to_plot]
    
    for param in param_cols:
        plt.figure(figsize=(10, 6))
        sns.scatterplot(
            data=df,
            x=param,
            y="value",
            hue="auc",
            palette="viridis",
            s=100 
        )
        clean_label = param.replace("params_", "").replace("_", " ").title()
        plt.title(f"{clean_label} vs Composite Score")
        plt.xlabel(clean_label)
        plt.ylabel("Composite Score")
        plt.grid(alpha=0.3)
        plt.show()

def run_optimization(real_data, discrete_columns, n_trials=20):
    study = optuna.create_study(
        direction="maximize",  
        sampler=optuna.samplers.TPESampler(seed=42),
        pruner=optuna.pruners.MedianPruner()
    )
    
    study.optimize(
        lambda trial: objective(trial, real_data, discrete_columns),
        n_trials=n_trials,
        show_progress_bar=True
    )

    print("\nBest trial metrics:")
    metrics = study.best_trial.user_attrs["metrics"]
    for k, v in metrics.items():
        print(f"{k.upper():<10}: {v:.4f}")
    
    visualize_study(study)
    return study

e:\qycache\anaconda\envs\LLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
import numpy as np

def auto_detect_discrete_columns(data, unique_ratio_threshold=0.05, unique_count_threshold=20):

    discrete_cols = []
    
    for col in data.columns:
        col_data = data[col].dropna()
        

        if len(col_data) == 0:
            continue
        

        dtype = col_data.dtype
        

        if dtype in [object, 'category', bool]:
            discrete_cols.append(col)
            continue

        if np.issubdtype(dtype, np.number):
            n_unique = col_data.nunique()
            total = len(col_data)

            if n_unique <= unique_count_threshold:
                discrete_cols.append(col)
                continue

            unique_ratio = n_unique / total
            if unique_ratio < unique_ratio_threshold:
                if np.issubdtype(dtype, np.integer):
                    discrete_cols.append(col)
                elif (col_data == col_data.astype(int)).all():
                    discrete_cols.append(col)
    
    return discrete_cols

if __name__ == "__main__":

    #real_path = "../synthcity-main/tutorials/covertype_preprocessed.csv"
    real_path = "../CTAB-GAN-main/Real_Datasets/Credit.csv"
    #real_path = "../CTAB-GAN-main/Real_Datasets/Adult3.csv"
    #real_path = '../CTGAN-main/CTGAN-main/examples/csv/train_clean.csv'
    real_data = pd.read_csv(real_path)
    
    discrete_cols = auto_detect_discrete_columns(real_data)
    print(discrete_cols)

In [ ]:
def smart_normalization(data, discrete_cols, threshold=0.1):

    continuous_cols = [col for col in data.columns if col not in discrete_cols]
    if not continuous_cols:
        return data.copy(), False

    needs_scaling = False
    for col in continuous_cols:
        col_mean = data[col].mean()
        col_std = data[col].std()

        if abs(col_mean) > threshold or not (1-threshold < col_std < 1+threshold):
            needs_scaling = True
            break
    
    if needs_scaling:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        data_scaled = data.copy()
        data_scaled[continuous_cols] = scaler.fit_transform(data[continuous_cols])
        return data_scaled, True
    else:
        return data.copy(), False

In [ ]:
#discrete_cols = []
study = run_optimization(real_data_normalized, discrete_cols, n_trials=20)

best_params = study.best_params
print("\nOptimal Parameters:", best_params)

[I 2025-02-22 13:38:31,294] A new study created in memory with name: no-name-fedb38c0-35c6-49cd-a945-38c5d8a714cc
  0%|          | 0/20 [00:00<?, ?it/s]e:\qycache\anaconda\envs\LLM\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/300 [00:00<?, ?it/s]
Hype-parameter:[dropout:0.21236203565420878,num_first:True,batch_first:True,num_heads:8,num_layers:3]
epoch!0
epoch!1
epoch!2
epoch!3
epoch!4
epoch!5
epoch!6
epoch!7
epoch!8
epoch!9
epoch!10
epoch!11
epoch!12
epoch!13
epoch!14
epoch!15
epoch!16
epoch!17
epoch!18
epoch!19
epoch!20
epoch!21
epoch!22
epoch!23
epoch!24
epoch!25
epoch!26
epoch!27
epoch!28
epoch!29
epoch!30
epoch!31
epoch!32
epoch!33
epoch!34
epoch!35
epoch!36
epoch!37
epoch!38
epoch!39
epoch!40
epoch!41
epoch!42
epoch!43
epoch!44
epoch!45
epoch!46
epoch!47
epoch!48
epoch!49
epoch!50
epoch!51
epoch!52
epoch!53
epoch!54
epoch!55
epoch!56
epoch!57
epoch!58
epoch!59
epoch!60
epoch!61
epoch!62
epoch!63
epoch!64
epoch!65
epoch!66
epoch!67
epoch!68
epoch!69
epoch!70
epoch!71
epoch!72
epoch!73
epoch!74
epoch!75
epoch!76
epoch!77
epoch!78
epoch!79
epoch!80
epoch!81
epoch!82
epoch!83
epoch!84
epoch!85
epoch!86
epoch!87
epoch!88
epoch!89
epoch!90
epoch!91
epoch!92
epoch!93
epoch!94
epoch!95
epoc

e:\qycache\anaconda\envs\LLM\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
e:\qycache\anaconda\envs\LLM\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
  5%|▌         | 1/20 [09:16<2:56:17, 556.69s/it]

[W 2025-02-22 13:47:47,983] Trial 0 failed with parameters: {'dropout': 0.21236203565420878, 'num_first': True, 'batch_first': True, 'num_layers': 3, 'num_heads': 8} because of the following error: The value nan is not acceptable.
[W 2025-02-22 13:47:47,986] Trial 0 failed with value nan.


e:\qycache\anaconda\envs\LLM\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


...

e:\qycache\anaconda\envs\LLM\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/300 [00:00<?, ?it/s]
Hype-parameter:[dropout:0.14346846162736693,num_first:False,batch_first:False,num_heads:8,num_layers:7]
epoch!0
epoch!1
epoch!2
epoch!3
epoch!4
epoch!5
epoch!6
epoch!7
epoch!8
epoch!9
epoch!10
epoch!11
epoch!12
epoch!13
epoch!14
epoch!15
epoch!16
epoch!17
epoch!18
epoch!19
epoch!20
epoch!21
epoch!22
epoch!23
epoch!24
epoch!25
epoch!26
epoch!27
epoch!28
epoch!29
epoch!30
epoch!31
epoch!32
epoch!33
epoch!34
epoch!35
epoch!36
epoch!37
epoch!38
epoch!39
epoch!40
epoch!41
epoch!42
epoch!43
epoch!44
epoch!45
epoch!46
epoch!47
epoch!48
epoch!49
epoch!50
epoch!51
epoch!52
epoch!53
epoch!54
epoch!55
epoch!56
epoch!57
epoch!58
epoch!59
epoch!60
epoch!61
epoch!62
epoch!63
epoch!64
epoch!65
epoch!66
epoch!67
epoch!68
epoch!69
epoch!70
epoch!71
epoch!72
epoch!73
epoch!74
epoch!75
epoch!76
epoch!77
epoch!78
epoch!79
epoch!80
epoch!81
epoch!82
epoch!83
epoch!84
epoch!85
epoch!86
epoch!87
epoch!88
epoch!89
epoch!90
epoch!91
epoch!92
epoch!93
epoch!94
epoch!95
ep

e:\qycache\anaconda\envs\LLM\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
e:\qycache\anaconda\envs\LLM\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 20/20 [4:00:42<00:00, 722.11s/it]


[W 2025-02-22 17:39:13,420] Trial 19 failed with parameters: {'dropout': 0.14346846162736693, 'num_first': False, 'batch_first': False, 'num_layers': 7, 'num_heads': 8} because of the following error: The value nan is not acceptable.
[W 2025-02-22 17:39:13,423] Trial 19 failed with value nan.

Best trial metrics:


ValueError: No trials are completed yet.